# Project: SVD Using Fixed Point Iterations

Jake Swartwout

## Intro

For this project, I was interested in learning more about SVD. We briefly went over it's use, but I was curious how one would implement it in code. So, I found this python implementation for it:
https://github.com/j2kun/svd

While looking at this package, I found that they also used fixed point iterations, which I found to be very interesting. I'll explain how later on in this report

This page specifically is the code I was looking at: https://github.com/j2kun/svd/blob/master/svd.py

While it is an entire collection of files, the useful portion is a single python file named svd.py

I didn't want to reupload someone else's code to github, so I put it in an alternate folder and had to phenagle to get in in here, but usually you would just do the basic

```python
from svd import svd
```

In [13]:
import sys
homePath = sys.path
sys.path.append('../svd')
from svd import svd
sys.path = homePath

## About the software

At a high level, this software performs the "Singular Value Decomposition" (SVD) of a matrix. This means that it is splitting any matrix A (it doesn't even need to be square) into 3 separate matrices. We represent it like this:

$$
A = U \Sigma V^T
$$

With the following properties:

$U$ and $V$ have orthonormal columns

$\Sigma$ is a diagonal matrix

This may seem familiar to diagonalization, as it does something similar. One of the main benefits however, is that our matrix A does not have to be square for SVD

## About the method

As we've seen in class, decomposing matrices into orthonormal bases is useful in reducing the solve time.

This is useful in making solving matrices easier. If we were to solve $Ax = b$, it would be useful to convert into $A = U \Sigma V^T$. The, we can use the property of U having orthonormal columns to simplify the expression.

We say that now $U\Sigma V^Tx = B$, and applying $U^T$ to both sides allows us to remove it from the left, leaving $\Sigma V^Tx = U^TB$. Then, if we solve this for $V^Tx$ (which would be easy, since $\Sigma$ is diagonal), we could finish and find x by applying V to the left of our solution.

So, it only takes one execution to find the values for the component matrices, and is much quicker to solve new equations once we already have the values.

The specific method to get these values, as stated by the author, is "An implementation of the greedy algorithm for SVD, using the power method for the 1-dimensional case." This means that we will perform the power method on our matrix A, one dimension at a time to get the final result. Dimension is a little misleading however, since it is not a column of the matrix, but rather a dimension of the transformation basis.

So, the code decomposes the matrix one eigenvector at a time. To find these vectors, it performs fixed point iterations on a random vector. Since our transformation stretches all points, a random vector (after applying this transformation and normalizing it) will end up pointing more in the direction of that eigenvalue. If we do this again and again, it will move more and more towards the eigenvector with the largest eigenvalue. We are then able to remove this from our original matrix, and do it again to find the lower level dimensions. The actual algorithm is more nuanced, such as using $A^TA$ rather than just A, but the idea remains the same.

## Performance

Unfortunately, I don't believe this method would be high in performance. Guessing the eigenvectors in this manner seems highly inefficient, and will force us to choose between higher accuracy and better runtime (as we choose how much we want it to converge before stopping). Beyond that, I'm sure that this type of algorithm is not well conditioned. But, this seems more of a demonstration than a full optimization, which makes sense that we wouldn't use this implementation. It was still interesting to read about nonetheless.

## Example usage

Here, I'll show a quick example of calling the function

In [31]:
A = [[15.,  6., 17., 18.],
     [ 7., 15., 14., 11.],
     [10.,  7., 14.,  8.],
     [10.,  4.,  6.,  2.],
     [14., 18., 17., 15.],
     [19., 10.,  8., 10.],
     [14., 15., 13., 17.]]
A = np.array(A)
A

array([[15.,  6., 17., 18.],
       [ 7., 15., 14., 11.],
       [10.,  7., 14.,  8.],
       [10.,  4.,  6.,  2.],
       [14., 18., 17., 15.],
       [19., 10.,  8., 10.],
       [14., 15., 13., 17.]])

In [21]:
sigma, u, v = svd(A)

converged in 5 iterations!
converged in 15 iterations!
converged in 17 iterations!
converged in 2 iterations!


It stores the single values in a one dimensional array, but can be turned back into a square one using `np.diag`

In [26]:
sigma

array([65.54751762, 11.87934328,  9.47498127,  6.37407772])

In [24]:
np.diag(sigma)

array([[65.54751762,  0.        ,  0.        ,  0.        ],
       [ 0.        , 11.87934328,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  9.47498127,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  6.37407772]])

In [27]:
u

array([[-0.43370825, -0.30227431,  0.75891686,  0.17328245],
       [-0.35570374,  0.56744182, -0.09446739, -0.16806248],
       [-0.30068402, -0.03273509,  0.24353264, -0.60915394],
       [-0.16952485, -0.35485975, -0.22784483, -0.49844102],
       [-0.48606016,  0.3219185 , -0.21111503, -0.0876953 ],
       [-0.35897388, -0.58494942, -0.4973935 ,  0.14462961],
       [-0.44844813,  0.11766757, -0.10922383,  0.5418561 ]])

In [28]:
v

array([[-0.5126244 , -0.45442206, -0.52701092, -0.5029675 ],
       [-0.7911014 ,  0.56901513,  0.21387742,  0.06809409],
       [-0.32250827, -0.68418055,  0.48901039,  0.43445855],
       [-0.08580918,  0.0402411 , -0.66135072,  0.74406496]])

And then we can recombine to check how well it did

In [32]:
recombined = u @ np.diag(sigma) @ v
recombined

array([[15.,  6., 17., 18.],
       [ 7., 15., 14., 11.],
       [10.,  7., 14.,  8.],
       [10.,  4.,  6.,  2.],
       [14., 18., 17., 15.],
       [19., 10.,  8., 10.],
       [14., 15., 13., 17.]])

But that's just because it's rounding. Here is how close the values actually are to the original:

In [40]:
diff = recombined - A
print(diff)

[[-3.55271368e-15  8.88178420e-16 -1.06581410e-14  1.06581410e-14]
 [-4.44089210e-15 -3.55271368e-15 -5.32907052e-15  8.88178420e-15]
 [-3.55271368e-15 -1.77635684e-15 -7.10542736e-15  7.10542736e-15]
 [-3.55271368e-15  0.00000000e+00 -3.55271368e-15  1.33226763e-15]
 [-3.55271368e-15  0.00000000e+00 -7.10542736e-15  1.24344979e-14]
 [-3.55271368e-15  0.00000000e+00 -5.32907052e-15  8.88178420e-15]
 [-3.55271368e-15 -3.55271368e-15 -7.10542736e-15  1.06581410e-14]]


In [41]:
np.max(diff), np.min(diff)

(1.2434497875801753e-14, -1.0658141036401503e-14)

In [42]:
np.linalg.norm(diff)

3.241851231905457e-14